# NLP for Finance — LLMs, Prompt Engineering, and Agents (LangGraph)

**Hands-on goals (1.5h):**

1. Call an LLM from Python.

2. See how prompts change behavior (prompt engineering).

3. Ground the LLM on finance text to reduce hallucinations.

4. Build a simple text tool over finance documents.

5. Wrap it in a LangGraph agent that decides when to call tools and how to answer.


In [ ]:
# %% Setup: imports & keys

# If running on Colab, you may need:
# !pip install openai langchain langgraph langchain-openai tiktoken --quiet

import os
from typing import List, Dict, Any

# --- OpenAI client (core LLM calls) ---
from openai import OpenAI

# We'll also use LangChain + LangGraph for the agent later
from langchain_openai import ChatOpenAI

client = OpenAI()

# Make sure OPENAI_API_KEY is set in your environment before running
assert os.getenv("OPENAI_API_KEY"), "Please set OPENAI_API_KEY in your environment."


In [ ]:
# %% Simple LLM helper

def llm_text(
    prompt: str,
    model: str = "gpt-4.1-mini",
    temperature: float = 0.2,
) -> str:
    """
    Minimal helper for single-turn prompts.
    """
    resp = client.responses.create(
        model=model,
        input=prompt,
        temperature=temperature,
    )
    # responses.create returns multiple output items; use the text helper:
    return resp.output_text

def llm_chat(
    messages: List[Dict[str, str]],
    model: str = "gpt-4.1-mini",
    temperature: float = 0.2,
) -> str:
    """
    Chat-style wrapper using role/content messages.
    """
    resp = client.responses.create(
        model=model,
        input=messages,
        temperature=temperature,
    )
    return resp.output_text


In [ ]:
# %% Load example finance documents (text only)

# In practice, you'd load from files/CSV. For the workshop, keep it simple:
DOCS = [
    """
    Our business is subject to a number of risks, including fluctuations in interest rates,
    exposure to foreign exchange rate volatility, and increased competition in our core markets.
    We may not be able to maintain our current margins if macroeconomic conditions deteriorate.
    """,
    """
    Management is optimistic about revenue growth in the coming fiscal year, driven by strong demand
    in our consumer lending segment. However, regulatory changes under consideration could increase
    capital requirements and negatively impact returns.
    """,
]

print(f"Loaded {len(DOCS)} finance documents.")


## 1. Prompt Engineering Basics: Role & Style

In this section:

- Call the LLM directly.

- Change **role** and **style** instructions.

- See how answers differ for the same question.


In [ ]:
# %% Prompt engineering: role & style

text = DOCS[0]

base_question = "What risks should an equity investor pay attention to in this document?"

prompt_equity = f"""
You are a senior equity research analyst.

Document:

{text}

Task:

- Answer the question below in 3 bullet points.

- Be concise and use plain English.

Question:

{base_question}
"""

print(llm_text(prompt_equity))


### 👉 Student TODO

1. Copy the previous cell.

2. Change the **role** and **audience**, e.g.:

   - "You are a chief risk officer."

   - "Explain to a first-year finance student."

3. Run and compare the tone and focus.


In [ ]:
# %% Prompt engineering: structured JSON output

prompt_json = f"""
You are an analyst. Extract *up to 5* key risks from the document.

Document:

{text}

Return a JSON object with this format only:

{{
  "risks": [
    {{"label": "...", "quote": "...", "reason": "..."}}
  ]
}}

- "label": short category of the risk (e.g., "interest rate risk").

- "quote": one short snippet from the document.

- "reason": why this matters to investors.
"""

print(llm_text(prompt_json))


## 2. Grounding: Using the Document vs. Hallucinating

We'll contrast:

- asking a generic question with *no* context

- vs. forcing the model to use the **specific document** only.


In [ ]:
# %% Grounding demo: with vs without document

question = "What regulatory risks are mentioned?"

# 1) No context – model may hallucinate
print("=== NO CONTEXT ===")
print(llm_text(question))

# 2) With context – grounded answer
prompt_grounded = f"""
You MUST answer only using the document below.
If the document does not mention a risk type, say "Not specified in the document."

Document:

{text}

Question:

{question}
"""

print("\n=== WITH DOCUMENT (GROUNDED) ===")
print(llm_text(prompt_grounded))


### 👉 Student TODO

Write a prompt that:

- Uses **only** the document.

- Extracts all sentences related to "competition".

- Returns a list of sentences in JSON: `{"sentences": ["...", "..."]}`.


In [ ]:
# %% 3. Text Tools: simple Python functions over finance text

import re

def split_sentences(text: str) -> List[str]:
    """
    Very naive sentence splitter, good enough for workshop demo.
    """
    # split on '.', '!' or '?'
    parts = re.split(r"[.!?]\s+", text.strip())
    # filter empty
    return [p.strip() for p in parts if p.strip()]

def find_sentences_with_keyword(text: str, keyword: str) -> List[str]:
    """
    Return sentences that contain the keyword (case-insensitive).
    """
    keyword_lower = keyword.lower()
    sentences = split_sentences(text)
    return [s for s in sentences if keyword_lower in s.lower()]

def find_risk_sentences(text: str) -> List[str]:
    """
    Very naive: sentences mentioning 'risk', 'uncertain', 'may not', etc.
    """
    patterns = ["risk", "uncertain", "may not", "could", "volatility"]
    sentences = split_sentences(text)
    hits = []
    for s in sentences:
        if any(p in s.lower() for p in patterns):
            hits.append(s)
    return hits

# quick test
print(find_risk_sentences(DOCS[0]))


In [ ]:
# %% Use tool results inside a prompt

sentences = find_risk_sentences(text)

tool_prompt = f"""
You are an analyst assistant.

I have extracted these 'risk-related' sentences from a filing:

{chr(10).join('- ' + s for s in sentences)}

Task:

- Summarize the main 2–3 categories of risk.

- One short bullet per category.
"""

print(llm_text(tool_prompt))


## 4. Building an Agent with LangGraph

We'll use:

- LangChain's `ChatOpenAI` as the model wrapper.

- LangChain `@tool` decorators for Python tools.

- LangGraph's `StateGraph` to define the agent workflow.

The agent will:

1. Receive a user question + a document.

2. LLM decides: answer directly vs call a tool.

3. If tool is called, we run the Python function.

4. LLM uses the tool result to produce the final answer.


In [ ]:
# %% LangGraph setup: model + tools

from langchain.tools import tool
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import AnyMessage, HumanMessage, SystemMessage, ToolMessage
from typing_extensions import TypedDict, Annotated
import operator

# LangChain chat model using OpenAI
lc_model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.1,
)

# --- Define tools using the same logic as before ---

@tool
def extract_sentences_with_keyword(text: str, keyword: str) -> List[str]:
    """Extract sentences from the given document that contain the given keyword (case-insensitive)."""
    return find_sentences_with_keyword(text, keyword)

@tool
def extract_risk_sentences(text: str) -> List[str]:
    """Extract sentences that likely mention risk (e.g., 'risk', 'uncertain', 'may not', 'volatility')."""
    return find_risk_sentences(text)

tools = [extract_sentences_with_keyword, extract_risk_sentences]
tools_by_name = {t.name: t for t in tools}

# Bind tools to the model so it can choose to call them
model_with_tools = lc_model.bind_tools(tools)


In [ ]:
# %% Define agent state for LangGraph

class AgentState(TypedDict):
    messages: Annotated[List[AnyMessage], operator.add]
    llm_calls: int
    # You can add more fields (e.g., doc_id), but keep it minimal for the workshop.


In [ ]:
# %% Model node: LLM decides whether to call a tool

def llm_node(state: AgentState) -> Dict[str, Any]:
    """
    LLM node:

    - Receives current messages (including the user's question and document).

    - Decides whether to call a tool.

    - Returns a new LLM message (could include tool calls).
    """
    system_msg = SystemMessage(
        content=(
            "You are an NLP assistant for finance documents. "
            "You may call tools to extract sentences. "
            "Use tools when the user asks you to 'find', 'extract', or 'highlight' sentences. "
            "Otherwise, answer directly. "
            "Always stay grounded in the document content."
        )
    )
    result = model_with_tools.invoke([system_msg] + state["messages"])
    return {
        "messages": [result],
        "llm_calls": state.get("llm_calls", 0) + 1,
    }


In [ ]:
# %% Tool node: execute any tool calls

def tool_node(state: AgentState) -> Dict[str, Any]:
    """
    Execute any tool calls requested by the last LLM message
    and return ToolMessages with the observations.
    """
    last_msg = state["messages"][-1]
    results: List[ToolMessage] = []

    if not getattr(last_msg, "tool_calls", None):
        return {"messages": []}

    for tool_call in last_msg.tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]
        tool = tools_by_name[tool_name]
        observation = tool.invoke(tool_args)
        # Wrap observation in a ToolMessage
        results.append(
            ToolMessage(
                content=str(observation),
                tool_call_id=tool_call["id"],
            )
        )

    return {"messages": results}


In [ ]:
# %% Routing logic: should we call a tool or stop?

from typing import Literal

def should_continue(state: AgentState) -> Literal["tool_node", END]:
    """
    Decide whether to continue to the tool node or end.

    If the last LLM message has tool_calls, we go to tool_node.

    Otherwise, we end (and return the answer).
    """
    messages = state["messages"]
    last = messages[-1]
    if getattr(last, "tool_calls", None):
        return "tool_node"
    return END


In [ ]:
# %% Build and compile the LangGraph agent

agent_builder = StateGraph(AgentState)

# Add nodes
agent_builder.add_node("llm_node", llm_node)
agent_builder.add_node("tool_node", tool_node)

# Edges
agent_builder.add_edge(START, "llm_node")
agent_builder.add_conditional_edges(
    "llm_node",
    should_continue,
    ["tool_node", END],
)
agent_builder.add_edge("tool_node", "llm_node")

# Compile
agent = agent_builder.compile()


In [ ]:
# %% Helper to run a single agent turn

def run_agent(question: str, document: str):
    """
    Prepare messages and invoke the LangGraph agent.
    """
    user_content = (
        f"Here is a finance document:\n\n{document}\n\n"
        f"User question: {question}"
    )
    initial_state: AgentState = {
        "messages": [HumanMessage(content=user_content)],
        "llm_calls": 0,
    }

    final_state = agent.invoke(initial_state)

    print("=== Final messages ===")
    for m in final_state["messages"]:
        # pretty_print is available but we can just print content
        print(f"[{m.type}] {getattr(m, 'content', m)}")


In [ ]:
# %% Try out the agent

doc = DOCS[1]

questions = [
    "Summarize the main risks for investors.",
    "Find sentences that mention regulation or regulatory changes.",
    "Highlight any forward-looking statements or guidance.",
]

for q in questions:
    print("\n" + "#" * 80)
    print("Question:", q)
    run_agent(q, doc)


## 5. Extensions / Experiments

👉 Ideas for students:

1. Add a new tool, e.g.:

   - `extract_forward_looking(text: str)` — sentences with "expect", "anticipate", "guidance", "forecast".

2. Modify the system prompt in `llm_node` so the model:

   - Uses the new tool when appropriate.

   - Explains when it *chose not* to call a tool.

3. Add a constraint:

   - "If you are unsure, say 'Not specified in the document.'"

   and observe how behavior changes.
